# Szöveges fájlformátumok

---

(2021. 02. 12.)

Mittelholcz Iván

# 1. Plain text (TXT)

# 1.1. Plain text fájlok soronkénti feldolgozása (*filter*)

Séma, a `process()` függvény változtatásával kb. bármilyen, soronként kezelhető problémára jó (egyszerre csak egy sor és feldolgozása van a memóriában):

```py
inp = open('input.txt')
out = open('output.txt', 'w')

for line in inp:
    line = line.strip()
    if not line:
        continue
    res = process(line)
    out.write(res)

inp.close()
out.close()
```

In [ ]:
# rossz!!! (mind a bemenet, mind a kimenet egyszerre a memoriaban)

inp = open('input.txt')
text = inp.read()
inp.close()

res = []
for line in text.split('\n'):
    line = line.strip()
    if not line:
        continue
    res.append(process(line))

out = open('output.txt', 'w')
out.write('\n'.join(res))
out.close()

# 1.2. Reguláris kifejezések

## 1.2.1. Mi egy reguláris kifejezés (*regex*)?

*reguláris kifejezés*: stringek egy halmazát határozza meg (egy reguláris nyelvet), tartalmazhat literális karaktereket és műveleteket.

Műveletek:

- *Konkatenáció*: R1 és R2 reguláris kifejezés, ekkor R1R2 is regularis kifejezés és R1R2 = {ab : a eleme R1 és b eleme R2}.
- *Unió* (v. *alternáció*): R1 és R2 reguláris kifejezés, ekkor R1 unió R2 is regularis kifejezés és R1 unió R2 = {a : a eleme R1 vagy a eleme R2}.
- *Kleene csillag*: R reguláris kifejezés, ekkor R* is reguláris kifejezés, mely tartalmazza az üres stringet és R elemeinek tetszőleges konkatenációját.

Műveletek precedenciája: *csillag* > *konkatenáció* > *alternáció*

## 1.2.2. Szintaxis

### Pozícióra (nulla karakterre) illesztés

- `^`: string / sor elejére illeszkedik
- `$`: string / sor végére illeszkedik

### Egy karakterre illesztés

- `.`: bármilyen karakterre illeszkedik
- különleges karakterre azt iszképelve lehet illeszteni, pl. `\.` illeszkedik a .-ra.
- `x`: literális karakter, saját magára illeszkedik
- `[ ]`: a zárójelen belül felsorolt karakterek valamelyikére illeszkedik, pl. `[ab]` illeszkedik az `a` vagy a `b` karakterre, másra nem.
    - Megadható tartomány is, pl `[a-z]` illeszkedik az ASCII kisbetűkre, `[0-9]` pedig a számjegyekre.
    - Ha a kötőjelet is be akarjuk venni a felsorolt karakterek közé, akkor a felsorolás elejére vagy végére kell írni.
    - A szögletes zárójelen belül más karakterek elveszítik speciális jelentésüket, pl. `[.]` egy literális pontra illeszkedik, nem pedig bármire.
- `[^ ]` illeszkedik a zárójelen belül fel nem sorolt karakterek valamelyikére. Megadható tartomány is, pl. `[^A-Z0-9]` illeszkedik minden karakterre, ami nem ASCII nagybetű és nem is számjegy.

### Változó hosszúságú illesztések (mindig mohó)

- `|`: Alternáció, az előtte vagy az utána következő regex valamelyikére illeszkedik, pl. abcd|xyz illeszkedik abcd-re és xyz-re is. Alternációt lehatárolni zárójellel lehet, pl. `ab(cd|xy)z` illeszkedik az *abcdz* és az *abxyz* stringekre.
- `?`: nulla vagy egy az előző karakterből / csoportból
- `*`: nulla vagy bármennyi az előző karakterből / csoportból
- `+`: legalább egy az előző karakterből / csoportból
- `{m,n}`: minimum *m*, maximum *n* darab az előző karakterből / csoportból.
    - `{m,}` alakban csak a minimumot is megadhatjuk (a maximum ekkor bármennyi lehet, hasonlóan a `*`-hoz).
    - `{,n}` alakban csak a maximumot is megadhatjuk (a minimum ekkor nulla, hasonlóan a `*`-hoz)
    - `{m}`: pontosam *m* darab

### Különleges karakterek

- `\n`: új sor (new line)
- `\t`: TAB
- `\s`: whitespace karakterek
- `\S`: nem whitespace karakterek
- `\w`: szóalkotó karakterek (számjegyek, betűk és alulvonás)
- `\W`: nem szóalkotó karakterek
- `\d`: számjegyek (*digit*)
- `\D`: nem számjegyek

### Csoport és hivatkozás csoportra

- `()`: A zárójelen belüli kifejezés megnevezett csoport lesz, amire később hivatkozni lehet. Általában egymásba ágyazhatók, de nem fedhetnek át. A *(a.(.a))* illeszkedik pl. az abba stringre. Zárójelre hivatkozni backslash-sel lehet:  `\(` és `\)`
- `\n`: Hivatkozás egy csoportra (az *n* itt a csoport száma helyett áll). Pl. `(a.(.a)) \2 \1` illeszkedik az abba ba abba stringre.

Egy-egy reguláris nyelv általában sokféleképpen megadhatók regexekkel (pl. `a+` = `aa*`), nincs igazán jó egyszerűsítő módszer, ezért érdemes jól megírni a regexeket!

### Regexek a Python-ban

Python modul: `re`

Dokumentáció: <https://docs.python.org/3/library/re.html>

HOWTO: <https://docs.python.org/3/howto/regex.html#regex-howto>

A Python hajlamos a stringekben lecserélni dolgokat (pl. a \n-t sortörésre, stb.). Hogy ne kelljen mindent kiiszképelni (sőt, az iszképelő backslash-t is iszképelni), ezért minták megadásánál érdemes ún. *raw string*-eket használni. Ezt a string literál elé írt *r* betűvel lehet elérni, pl. `nyers = r'valami\n'`


# 1.3. Keresés

A `re.search(pattern, text)` függvény a megadott szövegben keresi a mintát. Ha van találat, akkor az első előfordulással tér vissza (ún. *match object*).

Példa:

```py
import re
m = re.search(r'(\w+) (\w+) (\d+)?', 'ritkán rikkant a rigó')
if m:
    print(m.group(0)) # a teljes illeszkedő szövegrész (nem kell zárójelezve legyen)
    print(m.group(1)) # 1-as csoport
    print(m.group(2)) # 2-es csopot
    print(m.group(3)) # 3-es csopot
    print(m.group(1,2)) # tuple az 1-es és a 2-es csoportból
```

A `re.findall(pattern, text)` függvény a megadott szövegben keresi a minta összes, nem átfedő előfordulását. Az előfordulások listájával tér vissza (ha nem voltak csoportok, akkor stringek listája, ha voltak, akkor tuple-ök listája).

Példa:

```py
m = re.findall(r'((\d+ )?\w+)', '100 forintnak 50 a fele')
print(m)
# [('100 forintnak', '100 '), ('50 a', '50 '), ('fele', '')]
```

# 1.4. Csere és törlés

A `re.sub(pattern, replace, text)` függvény lecseréli a minta előfordulásait a szövegben és a módosított szöveggel tér vissza.

A mintában lévő csoportokra `\szám` formában lehet hivatkozni a csere-szövegben.

```py
new = re.sub(r'(\d+)', r'\1 db', '10 rigó rikkant')
print(new)
```

# 1.5. Mohóság

Mohó operátorok: `*`, `+`, `?`

Lusta operátorok: `*?`, `+?`, `??`

```py
string = 'xaaaxaaax'
greedy = re.search(r'x.*x', string)
lazy = re.search(r'x.*?x', string)
print('mohó:', greedy.group())
print('lusta:', lazy.group())
```

# 1.6. Optimalizálás

A mintából mindig egy objektum generálodik, ez költésges folyamat. A `re.compile(pattern)` függvénnyel elmenthetjük ezt az objektumot, később számtalanszor újrahasznosíthatjuk. Példa:

```py
pat = re.compile(pattern)
result = pat.search(text)
# ugyan az, mint a result = re.search(pattern, text)
```

Rossz gyakorlat:

```py
for line in inp:
    result = re.findall(pattern, line)
```

Jó gyakorlat:

```py
p = re.compile(pattern)
for line in inp:
    result = p.findall(line)
```

# 1.7. Feladatok

- Írjunk egy függvényt, ami egy szöveget és egy pozitív egész számot vár paraméterként. A függvény adja vissza, hogy hány adott hosszúságú szó van a szövegben. Punktuációk sorozata nem számít szónak.
- Írjunk egy függvényt, ami egy stringről eldönti, hogy a római szám-e! A neve legyen `is_roman_num(string)`, visszatérési értéke pedig boolean ([wiki](https://hu.wikipedia.org/wiki/R%C3%B3mai_sz%C3%A1m%C3%ADr%C3%A1s)).

# 2. CSV / TSV

CSV: Coma Separated Values

TSV: TAB Separated Values

Egy sor = egy objektum, egy oszlop = egy tulajdonság, opcionálisan van egy fejléc, az oszlopok nevével.

Példa:

```txt
Név     Fajta   Kor
Pongó   dalmata 6
Foltos  dalmata 0
Ezredes bobtail 15
Nózi    vizsla  14
```

# 2.1. Olvasás

Modul: `csv` (a TSV fájlokhoz is ezt használjuk)

Dokumentáció: <https://docs.python.org/3/library/csv.html>

Példa: TSV sorok beolvasása listába (`process`: listát feldolgozó függvény)

```py
import csv

inp = open('input.tsv')
reader = csv.reader(inp, delimiter='\t')
for row in reader:
    process(row)
```

In [ ]:
import csv

class Kutya:
    def __init__(self, nev, fajta, kor):
        self.nev = nev
        self.fajta = fajta
        self.kor = kor

inp = open('101.tsv')
reader = csv.reader(inp, delimiter='\t')
kutyak = []
for row in reader:
    kutyak.append(Kutya(*row)) # kicsomagolas: Kutya([nev, fajta, kor]) --> Kutya(nev, fajta, kor)

for kutya in kutyak:
    print(kutya.nev, kutya.fajta, kutya.kor)

# 2.2. Írás

Példa: listák kiírása TSV fájlba (`data`: listák listája)

```py
import csv

out = open('output.tsv', 'w')
writer = csv.writer(out, delimiter='\t')
for row in data:
    writer.writerow(row)
```

In [ ]:
import csv

out = open('output.tsv', 'w')
writer = csv.writer(out, delimiter='\t')
for k in kutyak:
    writer.writerow([k.nev, k.kor, k.fajta])
    
out.close()

# 2.3. Feladatok

- Írjunk egy szkriptet, ami a `101.tsv` fájl alapján kiszámolja a kutyák átlagéletkorát.
- Írjunk egy szkriptet, ami tetszőleges szöveges fájlból szógyakoriságot számol, az eredményt egy fejléces TSV fájlba írja ki.

# 3. XML / HTML

Példa (l. <https://hu.wikipedia.org/wiki/XML>):

```xml
<Recept név="kenyér" elk_idő="5 perc" sütés_idő="3 óra">
  <cím>Egyszerű kenyér</cím>
  <összetevő mennyiség="3" egység="csésze">Liszt</összetevő>
  <összetevő mennyiség="10" egység="dekagramm">Élesztő</összetevő>
  <összetevő mennyiség="1.5" egység="csésze">Meleg víz</összetevő>
  <összetevő mennyiség="1" egység="teáskanál">Só</összetevő>
  <Utasítások>
    <lépés>Keverj össze minden összetevőt, aztán jól gyúrd össze!</lépés>
    <lépés>Fedd le ruhával és hagyd pihenni egy óráig!</lépés>
    <lépés>Gyúrd össze újra, aztán süsd meg a sütőben!</lépés>
  </Utasítások>
</Recept>
```

# 3.1. Szintaxis

- csak séma: jelölő nyelv, bármit leírhat (vs HMTL), nincsenek előre definiált tagek
- fa struktúra: egy gyökér elem, minden más az ő "gyereke", vagy a gyerekének a gyereke, sít.
- *elem*: minden, ami "fel van címkézve", pl.

  ```xml
  <...>liszt</...>
  ```

- *tag*: címkék az elemeken, kötelező lezárni őket, nem fedhetnek át, pl.

  ```xml
  <összetevő>...</összetevő>
  ```

- *attribútum*: plusz információk (kulcs-érték párok) a nyitótageken belül, pl.
  ```xml
  <... egység="csésze">...</...>
  ```

# 3.2. Beautifulsoup alapok

Dokumentáció: <https://www.crummy.com/software/BeautifulSoup/bs4/doc/>

Telepítés (Beautifulsoup és xml parser):

```sh
pip install beautifulsoup4 lxml
```

Leveskészítés stringből (xml):

```python
from bs4 import BeautifulSoup as BS
xml = '<összetevő mennyiség="3" egység="csésze">Liszt</összetevő>'
soup = BS(xml, 'xml')
```

Leves fájlból (xml):

```py
inp = open('recept.xml')
soup = BS(inp, 'xml')
print(soup.prettify())
```

# 3.3. Tagek

Első a fában: `soup.tag`

Példa

```python
print(soup.cím)
print(soup.összetevő)
```

Összes a fában: `soup.find_all(tag)` (listaszerű)

Példa:

```python
for osszetevo in soup.find_all('összetevő'):
    print(osszetevo.get_text())
```

# 3.4. Tagek (folytatás)

Tag szöveges tartalma (elem): `tag.get_text()`

Példa:

```python
for osszetevo in soup.find_all('összetevő'):
    print(osszetevo.get_text())
```

Tag attribútumai: `tag.attrs` (dict)

Példa:

```python
soup.Recept.attrs
```

Egy attribútum értéke: `tag['attribútum']` (str)

```python
soup.Recept['név']
```

# 3.5. Feladatok

Kiinduló pont a `recept.xml` fájl.

- Hány összetevő kell a kenyérhez?
- Mi a mértékegysége a második összetevőnek?
- Mi az utolsó utasítás?
- Hány teáskanál 'Só' kell a kenyérhez? (Tegyük fel, hogy nem tudjuk, hányadik összetevő a 'Só'!)

# Köszönöm a figyelmet!